# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Starter Code

## Part A | RDBMS Databases and `SQLite`

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [31]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-04-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [33]:
print db

In [7]:
a = "This is a " +\
"string"

In [8]:
a

'This is a string'

In [9]:
b = """
This is one line
This is another
"""

In [10]:
b

'\nThis is one line\nThis is another\n'

In [36]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    WHERE is_a_studio = 1
    LIMIT 10
;
''', con = db)

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,15161351,"157 Winfield St, San Francisco, CA",37742237,-122418547,1,...,1375.0,sqft,1742.0,sqft,1907.0
1,15164199,"501 Crescent Ave, San Francisco, CA",37735041,-122416045,1,...,960.0,sqft,1751.0,sqft,1905.0
2,119685072,"574 Natoma St APT 302, San Francisco, CA",37779014,-122409446,1,...,370.0,sqft,NaN,None,2012.0
3,15064640,"1047-1049 Lombard St, San Francisco, CA",37801889,-122418704,1,...,4102.0,sqft,3049.0,sqft,1948.0
4,15094548,"528 2nd Ave, San Francisco, CA",37778685,-122459484,1,...,2810.0,sqft,3049.0,sqft,1906.0
5,15077111,"1200 Gough St APT 22B, San Francisco, CA",37784820,-122423994,1,...,550.0,sqft,NaN,None,1966.0
6,15180636,"1069 Capp St, San Francisco, CA",37749612,-122416789,1,...,3140.0,sqft,3737.0,sqft,1900.0
7,15069477,"631 Ofarrell St APT 816, San Francisco, CA",37785240,-122415125,1,...,480.0,sqft,NaN,None,1930.0
8,15153753,"1311 Palou Ave, San Francisco, CA",37730765,-122385777,1,...,2645.0,sqft,4996.0,sqft,1923.0
9,15143034,"66-70 Landers St, San Francisco, CA",37766736,-122428085,1,...,3775.0,sqft,3123.0,sqft,1905.0


### `SELECT`

In [17]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,id,sale_price,sale_price_unit
0,15165953,650000.00,$
1,80749447,1.15,$M
2,15155751,665000.00,$
3,15143887,2.10,$M
4,15117639,1.35,$M
...,...,...,...
995,69819708,731000.00,$
996,15076156,5.53,$M
997,119685619,625000.00,$
998,15113584,895000.00,$


### `WHERE`

In [18]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,80749447,1.15
1,15143887,2.10
2,15117639,1.35
3,80743040,1.48
4,15134909,1.09
...,...,...
570,2101028916,2.45
571,15091950,1.75
572,89237766,1.16
573,15197693,1.36


> ### Here's let's convert the unit of `sale_price` from \$ to \$M

In [19]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
420,119685503,0.750
421,69819708,0.731
422,119685619,0.625
423,15113584,0.895


### `UNION`

> ### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [20]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `WITH`

In [21]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `JOIN`

In [24]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `WITH` (take 2)

In [46]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM baths_normalized_sale_prices
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `GROUP BY` and `COUNT`

> ### Let's compute the number of properties based on their number of bedrooms

In [30]:
pd.io.sql.read_sql(
'''
SELECT COUNT() AS count
    FROM properties
    GROUP BY beds
;
''', con = db)

,count
0,164
1,142
2,316
3,226
4,103
5,28
6,9
7,6
8,3
9,3


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bathrooms

In [28]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT baths, AVG(sale_price) AS mean_sale_price
    FROM baths_normalized_sale_prices
    GROUP BY baths
;
''', con = db)

,baths,mean_sale_price
0,NaN,1.739026
1,1.00,0.987656
2,1.10,1.420000
3,1.25,1.600000
4,1.50,1.223378
...,...,...
16,6.50,16.000000
17,7.00,0.999000
18,7.50,5.530000
19,8.00,13.100000


Note: `baths = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [54]:
#Dunno

> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [66]:
pd.io.sql.read_sql(
'''
SELECT x.id AS id_x, y.id AS id_y, x.address AS ad_x, y.address AS ad_y
    FROM properties AS x
    JOIN properties AS y ON x.latitude = y.latitude AND x.longitude = y.longitude
    WHERE x.id > y.id
;
''', con = db)

,id_x,id_y,ad_x,ad_y
0,119685526,119685188,"1902 Filbert St, San Francisco, CA","1902A Filbert St, San Francisco, CA"
1,15076711,15076699,"1 Daniel Burnham Ct APT 1404, San Francisco, CA","1 Daniel Burnham Ct APT 1202, San Francisco, CA"
2,79846666,79844263,"555 4th St UNIT 513, San Francisco, CA","555 4th St, San Francisco, CA"
3,80730869,69819664,"260 King St UNIT 817, San Francisco, CA","260 King St UNIT 1205, San Francisco, CA"
4,80730869,69819845,"260 King St UNIT 817, San Francisco, CA","260 King St UNIT 769, San Francisco, CA"
...,...,...,...,...
103,111931130,111930969,"860 Waller St APT 1, San Francisco, CA","860 Waller St APT 2, San Francisco, CA"
104,2100850402,125156575,"1915 Diamond St, San Francisco, CA","1915 Diamond St, San Francisco, CA"
105,61288454,60825591,"77 Dow Pl APT 507, San Francisco, CA","77 Dow Pl APT 500, San Francisco, CA"
106,67395015,63107226,"8400 Oceanview Ter APT 218, San Francisco, CA","3981 Alemany Blvd APT 206, San Francisco, CA"
